# 3D Diffusion Equation in Cylindrical Coordinates

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns
import numba
import astropy.units as u
from IPython.display import HTML

sns.set_context(context='notebook',font_scale=1.5)

%matplotlib inline

We want to solve the 3D heat equation on a finite cylinder. The relevant PDE is,
$$
\frac{1}{\alpha}\frac{\partial T}{\partial t} = \frac{1}{r}\frac{\partial}{\partial r}\left(r\frac{\partial T}{\partial r}\right) + \frac{1}{r^2}\frac{\partial^2 T}{\partial \phi^2} + \frac{\partial^2 T}{\partial z^2},
$$
subject to the boundary conditions,
$$
T(r,\phi,z,0)=T_0 \\
T(R,\phi,z,t)=T_a \\
T(r,\phi,0,t)=T_a \\
t(r,\phi,L,t)=T_a \\
$$
We could also include an additional boundary condition on $r,\phi$ to account for uneven heating around the outside.

We need to discretize this equation in order to solve it numerically. Using a forward-differencing scheme, this will look like,
$$
\frac{1}{\alpha}\frac{T^{n+1}_{ijk}-T^n_{ijk}}{\Delta t} = \frac{r_{i+1/2}(\frac{T^n_{i+1,j,k} - T^n_{ijk}}{\Delta r}) - r_{i-1/2}(\frac{T^n_{ijk} - T^n_{i-1,j,k}}{\Delta r})}{r_i\Delta r} + \frac{T^n_{i,j+1,k} - 2T^n_{ijk} + T^n_{i,j-1,k}}{r_i^2\Delta\phi^2} + \frac{T^n_{i,j,k+1} - 2T^n_{ijk} + T^n_{i,j,k-1}}{\Delta z^2}
$$

In [6]:
# in meters
radius = 8.0e-2
length = 25.0e-2
delta_phi = 0.1
delta_z = 1.0e-2
delta_r = 0.1e-2
delta_t = 0.5
r = np.arange(5e-3,radius+delta_r,delta_r)
phi = np.arange(0,2.0*np.pi+delta_phi,delta_phi)
z = np.arange(0.0,length+delta_z,delta_z)

In [7]:
#thermal diffusivity constant, using the thermal conductivity and specific heat capacity of cow muscle 
#and the density of human muscle
alpha = 0.42/(1.0599e+3*3.52e+3)

We need to check that we are obeying the CFL stability condition,
$$
\frac{\alpha\tau}{h^2}\leq\frac{1}{4}
$$

In [8]:
if np.max(alpha*delta_t/(np.array([delta_r,np.min(r)*delta_phi,delta_z])**2)) > 0.25:
    delta_t = np.min(np.array([delta_r,np.min(r)*delta_phi,delta_z]))**2/alpha*0.25

Set some of the boundary and initial conditions.

In [9]:
def intial_temperature_distribution():
    return 293.0
def outer_temperature_distribution(phi,z):
    air_temp = 491.0
    surface_temp = 691.0
    return np.outer((surface_temp - air_temp)*np.exp(-(phi - np.pi)**2/((np.pi/6)**2)) + air_temp,np.ones(len(z)))
def endcaps_temperature_distribution():
    return 491.0

In [10]:
solutions = []
time = []
cylinder = np.ones([len(r),len(phi),len(z)])*intial_temperature_distribution()
old_cylinder = cylinder
cur_time = 0.0
solutions.append(np.copy(cylinder))
time.append(cur_time)
total_time = 7200

In [11]:
@numba.jit
def solve_heat_eq(cylinder,old_cylinder,r,phi,z,delta_r,delta_phi,delta_z):
    for i in range(1,len(r)-1):
        for j in range(1,len(phi)-1):
            for k in range(1,len(z)-1):
                r_term_1 = (r[i+1] + r[i])/2.0*(old_cylinder[i+1,j,k] - old_cylinder[i,j,k])/delta_r
                r_term_2 = (r[i-1] + r[i])/2.0*(old_cylinder[i,j,k] - old_cylinder[i-1,j,k])/delta_r
                r_term = (r_term_1 - r_term_2)/(r[i]*delta_r)
                phi_term = (old_cylinder[i,j+1,k] - 2.*old_cylinder[i,j,k] + old_cylinder[i,j-1,k])/(r[i]**2*delta_phi**2)
                z_term = (old_cylinder[i,j,k+1] - 2.*old_cylinder[i,j,k] + old_cylinder[i,j,k-1])/(delta_z**2)
                cylinder[i,j,k] = old_cylinder[i,j,k] + alpha*delta_t*(r_term + phi_term + z_term)
    return cylinder

In [12]:
while cur_time < total_time:
    #update time
    cur_time += delta_t
    #print('time = {}'.format(cur_time))
    cylinder = solve_heat_eq(cylinder,old_cylinder,r,phi,z,delta_r,delta_phi,delta_z)
    #set boundary conditions
    #r
    cylinder[0,:,:] = cylinder[1,:,:]
    cylinder[-1,:,:] = outer_temperature_distribution(phi,z)
    #phi
    cylinder[:,0,:] = cylinder[:,1,:]
    cylinder[:,-1,:] = cylinder[:,1,:]
    #z
    cylinder[:,:,0] = endcaps_temperature_distribution()
    cylinder[:,:,-1] = endcaps_temperature_distribution()
    #save time and solution
    solutions.append(np.copy(cylinder))
    time.append(cur_time)
    old_cylinder = cylinder

In [13]:
def make_frame(fig,sol,time):
    ax = fig.add_subplot(122,polar=True)
    ax2 = fig.add_subplot(121)
    im = ax.pcolormesh(PHI,R,
                       ((sol[-1][:,:,13].T)*u.K).to(u.imperial.deg_F,
                                                        equivalencies=u.equivalencies.temperature()).value,
                       vmin=32,vmax=500,
                      cmap=matplotlib.colors.ListedColormap(sns.color_palette('coolwarm',n_colors=1000))
              )
    cbaxes = fig.add_axes([0.53, 0.18, 0.4, 0.03]) 
    cb = fig.colorbar(im,cax=cbaxes,orientation='horizontal')
    ax.set_rticks([])
    ax.set_thetagrids([])
    ax.set_rmax(radius)
    ax.set_rmin(r[0])
    ax.set_title(r'$t={t:.1f}$ min'.format(t=(time[-1]*u.s).to(u.minute).value))
    cb.set_label(r'$T$ ($^{\circ}$F)')
    ax2.plot(time,((np.array([s[0,0,13] for s in sol])*u.K).to(u.imperial.deg_F,
                                                        equivalencies=u.equivalencies.temperature()).value))
    ax2.set_xlabel(r'$t$ (s)')
    ax2.set_ylabel(r'$T(r=0)$ ($^{\circ}$F)')
    ax2.set_xlim([0,total_time])
    ax2.set_ylim([60,200])

In [14]:
R,PHI = np.meshgrid(r,phi)
fig = plt.figure(figsize=(12,8))
for i in range(len(time)):
    if time[i]%60==0:
        make_frame(fig,solutions[:i+1],time[:i+1])
        #plt.tight_layout()
        plt.savefig('temp_figs/frame_{num:06d}.pdf'.format(num=i))
        fig.clf()

In [15]:
%%bash
convert -delay 10 -loop 0 temp_figs/*.pdf tenderloin_cooking.gif

In [16]:
HTML('<img src="tenderloin_cooking.gif"/>')